## Backtrader

[Backtrader](https://www.backtrader.com/) 是一個基於 Python 語言的進行自動化回溯測試的平臺。可以新增自定義的指標和交易策略,提高對交易系統回測的效率，這個工具可以匯入自己的行情資料檔案，也可以新增自定義的指標，測試結束後能顯示指標和行情圖表，而且可以對指標的不同引數設定進行批量測試！

如果有興趣還可以跟世界上所有投資客在 [官方論壇](https://community.backtrader.com/) 交流

## 安裝及引入方式

In [ ]:
!pip install backtrader backtrader_plotting

In [ ]:
import backtrader as bt
from datetime import datetime
import matplotlib
# 互動圖表
from backtrader_plotting import Bokeh
# Tradimo 或 Blackly
from backtrader_plotting.schemes import Blackly

In [ ]:
%matplotlib inline

### 設定運行環境

In [ ]:
cerebro = bt.Cerebro()

### 設定持有資產

1e5 就是 1 * 10 ^ 5

In [ ]:
cerebro.broker.setcash(1e5)

### 設定交易手續費(經理人傭金、平台手續費)

以下為在交易 Euro Stoxx 50 ETF 的交易傭金設定

In [ ]:
cerebro.broker.setcommission(
    # 如果有設定 margin ，則此傭金為固定數量，反之則為百分比 % 數
    # 此處則為買、賣固定 2.0 歐元
    commission=2.0,
    
    # 當需要模擬保證金交易(期貨、合約交易)等操作時需要使用
    # 如果這個參數是 0 ，則佣金為百分比 % 數
    margin=2000.0,
    
    # 在進行期貨等交易時可以設定的槓桿倍數
    mult=10.0,
    
    # 當需要同時開啟多筆交易時可以設定，會以名稱區隔交易績效
    name="Eurostoxxx50"
)


### 設定交易槓桿倍率

In [ ]:
# 固定購買 20 股
cerebro.addsizer(bt.sizers.SizerFix, stake=20)

### 加入分析指標 (選用)

可自行設定 _name 呼叫參數，例如設定為 'SharpeRatio' 則可透過 analyzers.SharpeRatio.get_analysis() 取得更細節的分析

In [ ]:
# 交易分析 (策略勝率)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='TradeAnalyzer')

# 交易基本統計分析
cerebro.addanalyzer(bt.analyzers.PeriodStats, _name='PeriodStats')

# 回落統計
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')

# 期望獲利/標準差 System Quality Number
cerebro.addanalyzer(bt.analyzers.SQN, _name='SQN')

# 夏普指數 
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')

### 引入資料 - 可以是 [YAHOO Finance](https://finance.yahoo.com/) 中下載的 CSV

In [ ]:
data = bt.feeds.YahooFinanceData(
    # 股票資料位置
    dataname='../input/quantitative-trading/TSLA.csv',
    
    # 資料起始日
    fromdate=datetime(2010, 7, 1),
    
    # 資料結束日
    todate=datetime(2020, 1, 1)
)


### 將資料引入環境

In [ ]:
cerebro.adddata(data)

### 運行環境

In [ ]:
print('投資 > 起始資產 %.2f 💲' % cerebro.broker.getvalue())
cerebro.run()
print('投資 > 結束資產 %.2f 💲' % cerebro.broker.getvalue())

### 繪製圖表

In [ ]:
# cerebro.plot()

### 繪製互動圖表

In [ ]:
# b = Bokeh(style='bar', plot_mode='single', scheme=Blackly())
# cerebro.plot(b)

## 策略格式

---

交易策略的開發非常簡單，只需先繼承 bt.Strategy，再調用幾項類別函數即可 [詳細使用文件](https://www.backtrader.com/docu/strategy/)

In [ ]:
# 1. 繼承 Backtrader
class BuyAndHold(bt.Strategy):
    # 2. 可以設定這個策略的可變參數
    params = (('slow', 15), ('fast', 5))
    
    # 3. 建構子：初始化策略，有些策略需要有歷史資料評估，就需要在此設定
    def __init__(self):
        pass
    
    # 在策略開始時調用
    def start(self):
        pass
    
    # 紀錄交易訊息，可以額外設定是否顯示紀錄、日期等，像一般我們改寫函數即可，非常直覺
    def log(self, txt):
        pass
    
    # 在比對完最新的收盤價格後調用
    def next(self):
        
        # 買入函數
        buy_info = self.buy(
            # 交易數量，預設可以由 getsizer 取得
            size=10,
            
            # 如果需要設定限價單的話，可以設定這個參數
            plimit=100
        )
        
        # 賣出函數
        sell_info = self.sell(
            # 交易數量，預設可以由 getsizer 取得
            size=10,
            
            # 如果需要設定限價單的話，可以設定這個參數
            plimit=100
        )
        
        pass
    
    # 在策略結束後調用
    def stop(self):
        pass
    
    # 在策略放置訂單後調用，取得訂單資訊
    def notify_order(self, order):
        pass
    
    # 在策略交易後調用，取得交易資訊
    def notify_trade(self, trade):
        pass